In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
from tqdm import tqdm

# os.system('wget %s' % xl_links[0])

In [ ]:
# url = 'https://ontobee.org/'

# source_code = requests.get(url)
# plain_text = source_code.content
# soup = BeautifulSoup(plain_text, "html.parser")
# links = soup.findAll('a', {'title': 'Excel XLSX File'})
# print(len(links))
# xl_links = [link.get('href') for link in links]

# # extract string between two strings, in this case, 'format=' and '&'
# file_names = [x.split('https://ontobee.org/listTerms/')[1].split(r'?format=')[0] for x in xl_links]

# files = list(zip(xl_links, file_names))

In [ ]:
# track files that could not be read in case needed later
unread_files = []

# download files
# for x in tqdm(files, leave = False): 
#     try: 
#         resp = requests.get(x[0])
#         path = ''.join(['/home/ubuntu/work/backup/terms/', f'{x[1]}.xlsx'])
#         if os.path.exists(path) != True:
#             with open(path, 'wb') as f:
#                 f.write(resp.content)
#     except: 
#         print("Could not read: ", x[0])
#         unread_files.append(x[0])

In [72]:
from dask import dataframe as dd
import dask
from dask.delayed import delayed
from dask.diagnostics import ProgressBar
from glob import glob

In [68]:
@dask.delayed
def try_to_read(file): 
    """ Try to read in file, if not, return empty dataframe """
    try: 
        df = pd.read_excel(file, engine='openpyxl', dtype=str).astype(str)
        return df
    except: 
        return pd.DataFrame(columns = ['Term IRI', 'Term label', 'Parent term IRI', 'Parent term label','Alternative term', 'Definition']).astype(str)

In [69]:
xl_files = glob('/home/ubuntu/work/backup/terms/*.xlsx')

output = []

for x in xl_files:
    parts = dask.delayed(try_to_read)(x)
    # filter_df = dask.delayed(get_techniques)(parts)
    output.append(parts)

# convert to a single dataframe
df_total = dd.from_delayed(output)

# df_total.visualize()

with ProgressBar():
    ddf = df_total.compute()

In [87]:
# create new column names with camel case
from re import sub

def camel_case(s):
  s = sub(r"(_|-)+", " ", s).title().replace(" ", "")
  return ''.join([s[0].lower(), s[1:]])

new_headers = [camel_case(x) for x in ddf.columns.tolist()]
col_map = dict(zip(ddf.columns.tolist(), new_headers))

In [91]:
def to_camel_case(text):
    text = text.replace("-", " ").replace("_", " ")
    words = text.split()
    return "".join([w.capitalize() if w != words[0] else w for w in words])

In [92]:
[to_camel_case(x) for x in ddf.columns.tolist()]

['termIri',
 'termLabel',
 'parentTermIri',
 'parentTermLabel',
 'alternativeTerm',
 'definition']

In [90]:
# rename columns
ddf.rename(columns=col_map, inplace=True)

In [97]:
def get_techniques(df): 
    """ Filter for techniques """
    df = df.loc[df['parentTermLabel'].str.contains('assay', case=False, na=False), ['termLabel', 'parentTermLabel', 'alternativeTerm', 'definition']].reset_index(drop=True)
    return df

In [98]:
# pattern = 'assay'
# ddf[ddf['Parent term label'].str.contains(pattern, case = False, na=False)]

# filter dataframe to only include assay terms. CSV will be uploaded to postgres DB
csv_path = '/home/ubuntu/work/backup/terms/assay_terms.csv'
get_techniques(ddf).to_csv(csv_path, index=False)

# Upload terms to database

In [99]:
import yaml
from sqlalchemy import create_engine, text, MetaData, Table, Column, Integer, String

# set up
with open("/home/ubuntu/work/therapeutic_accelerator/config/main.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
    
with open("/home/ubuntu/work/therapeutic_accelerator/config/keys.yaml", "r") as f:
    keys = yaml.load(f, Loader=yaml.FullLoader)

# Create engine to connect to database
engine = create_engine(f'postgresql://postgres:{keys["postgres"]}@{config["database"]["host"]}:5432/postgres')

In [100]:
# Read header from CSV to create columns
from csv import DictReader

with open(csv_path, 'r') as f:
    d_reader = DictReader(f)

    #get fieldnames from DictReader object and store in list
    headers = d_reader.fieldnames
    
headers

['termLabel', 'parentTermLabel', 'alternativeTerm', 'definition']

Delete old table if necessary to replace with new table

In [108]:
table_name = 'assayterms' # should be all lower case to avoid postgres issues

In [111]:
delete_table = True

if delete_table == True:
    sql = text(f''' 
        DROP TABLE IF EXISTS {table_name};
    ''')

    with engine.connect() as conn: 
        query = conn.execute(sql)

In [112]:
# Create Table in DB first before uploading
metadata_obj = MetaData()

# Create abstracts metadata
abstracts = Table(
    table_name,
    metadata_obj,
    Column("termLabel", String, nullable = True), # has to be integer so that dask can partition the table
    Column("parentTermLabel", String, nullable=True),
    Column("alternativeTerm", String, nullable=True),
    Column("definition", String, nullable=True)
)

metadata_obj.create_all(engine)

Has to be run in the terminal because you have to supply the password  
```psql --host=database-1.cuaho2dof33c.us-east-1.rds.amazonaws.com \
--port=5432 --username=postgres --password \
--dbname=postgres

In [104]:
# Upload CSV to DB
## Copy and paste the output below into the postgres terminal. 
## HEADER true means that the csv file contains the header. Number of columns in csv has to match exactly with DB columns
print(rf"\copy {table_name} FROM {csv_path} WITH (FORMAT CSV, HEADER true, DELIMITER ',');")

\copy assayTerms FROM /home/ubuntu/work/backup/terms/assay_terms.csv WITH (FORMAT CSV, HEADER true, DELIMITER ',');
